In [1]:
from pyomo_orm.core.database import Session, bind_engine
from pyomo_orm.core.utils import as_dataframe

from models import *

In [2]:
bind_engine('sqlite:///diet.sqlite')

Engine(sqlite:///diet.sqlite)

In [3]:
from pyomo_orm.core.problems import BaseProblem
from pyomo_orm.core.wrappers import *
infinity = float('inf')

In [4]:
from pyomo.environ import NonNegativeReals, NonNegativeIntegers

class DietProblem(BaseProblem):
    # Sets
    foods = ORMSet(
        model=Food,
        from_attr='id',
        indexed_by=None,
        queryset=Food.query().filter(Food.problem_run_id == None)
    )
    nutrients = ORMSet(
        model=Nutrient,
        from_attr='id',
        indexed_by=None,
        queryset=Nutrient.query().filter(Nutrient.problem_run_id == None)
    )

    # Params
    cost = ORMParam(
        'foods',
        model=Food,
        from_attr='cost'
    )
    volume_per_serving = ORMParam(
        'foods',
        model=Food,
        from_attr='volume_per_serving'
    )
    amount = ORMParam(
        'foods',
        'nutrients',
        model=FoodNutrientAmount,
        from_attr='amount',
        indexed_by=['food_id', 'nutrient_id']
    )
    nutrient_lower_bound = ORMParam(
        'nutrients',
        model=Nutrient,
        from_attr='lower_bound',
        within=NonNegativeReals,
        default=0.0
    )
    nutrient_upper_bound = ORMParam(
        'nutrients',
        model=Nutrient,
        from_attr='upper_bound',
        within=NonNegativeReals,
        default=infinity
    )
    
    volume_max = ORMParam(
        model=Constant,
        from_attr='value',
        indexed_by=None,
        queryset=Constant.query().filter(
            Constant.problem_run_id == None, 
            Constant.constant_name == 'max_volume'
        ),
        within=NonNegativeReals
    )
    
    # Vars
    amount_in_diet = ORMVar(
        'foods',
        model=Food,
        from_attr='amount_in_diet',
        within=NonNegativeIntegers
    )
    
    @orm_constraint('nutrients')
    def nutrient_lower_constraint(m, n):
        value = sum(m.amount[f, n] * m.amount_in_diet[f] for f in m.foods)
        return (m.nutrient_lower_bound[n] <= value)
    
    @orm_constraint('nutrients')
    def nutrient_upper_constraint(m, n):
        value = sum(m.amount[f, n] * m.amount_in_diet[f] for f in m.foods)
        return (value <= m.nutrient_upper_bound[n])
    
    @orm_constraint()
    def volume_constraint(m):
        return sum(m.volume_per_serving[f] * m.amount_in_diet[f] for f in m.foods) <= m.volume_max
    
    @orm_objective()
    def total_cost(m):
        return sum(m.cost[f] * m.amount_in_diet[f] for f in m.foods)


In [5]:
diet = DietProblem(
    name='Diet Problem', 
    description="""
From a selection of foods curate a meal plan that meets nutrient requirements at lowest cost
"""
)

In [6]:
diet.define_problem()

In [7]:
diet.pyomo_model.volume_max

In [8]:
diet.create_instance()

In [9]:
diet.create_solver()

<Plugin CBCSHELL 'cbc'>

In [10]:
diet.solve()

{'Problem': [{'Name': 'unknown', 'Lower bound': 15.05, 'Upper bound': 15.05, 'Number of objectives': 1, 'Number of constraints': 10, 'Number of variables': 10, 'Number of nonzeros': 77, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'User time': -1.0, 'Termination condition': 'optimal', 'Error rc': 0, 'Time': 0.10593223571777344}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [11]:
diet._pyomo_orm_instance_components

{<Amount: 2.0 of Iron in Orange Juice,
 <Food: Orange Juice>,
 <Amount: 20.0 of Iron in Cheeseburger,
 <Food: Lowfat Milk>,
 <Amount: 26.0 of Carbohydrates in Fries,
 <Amount: 3.0 of Protein in Fries,
 <Food: Sausage Biscuit>,
 <Food: Fries>,
 <Amount: 30.0 of Calcium in Cheeseburger,
 <Nutrient: Carbohydrates>,
 <Amount: 24.0 of Protein in Ham Sandwich,
 <Food: Chicken Sandwich>,
 <Amount: 8.0 of Iron in Chicken Sandwich,
 <Food: Fish Sandwich>,
 <Nutrient: Protein>,
 <Amount: 30.0 of Calcium in Lowfat Milk,
 <Constant: max_volume = 75.0>,
 <Food: Hamburger>,
 <Amount: 4.0 of Vitamin C in Lowfat Milk,
 <Nutrient: Calories>,
 <Food: Ham Sandwich>,
 <Amount: 20.0 of Calcium in Sausage Biscuit,
 <Amount: 110.0 of Calories in Lowfat Milk,
 <Amount: 10.0 of Vitamin C in Ham Sandwich,
 <Food: Cheeseburger>,
 <Amount: 6.0 of Vitamin C in Cheeseburger,
 <Amount: 25.0 of Calcium in Hamburger,
 <Amount: 4.0 of Vitamin A in Sausage Biscuit,
 <Amount: 2.0 of Calcium in Orange Juice,
 <Amount: 15.

In [12]:
from pyomo_orm.core.utils import as_dataframe

In [13]:
as_dataframe(Food.query())

,id,name,cost,volume_per_serving,amount_in_diet,problem_run_id
0,1,Cheeseburger,1.84,4.0,None,3
1,2,Ham Sandwich,2.19,7.5,None,3
2,3,Hamburger,1.84,3.5,None,3
3,4,Fish Sandwich,1.44,5.0,None,3
4,5,Chicken Sandwich,2.29,7.3,None,3
5,6,Fries,0.77,2.6,None,3
6,7,Sausage Biscuit,1.29,4.1,None,3
7,8,Lowfat Milk,0.60,8.0,None,3
8,9,Orange Juice,0.72,12.0,None,3


In [14]:
diet.current_problem_run.save()

In [15]:
from pyomo_orm.core.models import ProblemRun, ProblemDetail

In [16]:
as_dataframe(ProblemDetail.query())

,id,name,description,version
0,1,Diet Problem,\nFrom a selection of foods curate a meal plan...,
1,2,Diet Problem,\nFrom a selection of foods curate a meal plan...,
2,3,Diet Problem,\nFrom a selection of foods curate a meal plan...,


In [17]:
as_dataframe(ProblemRun.query())

,id,problem_details_id
0,1,1
1,2,2
2,3,3
